In [1]:
%load_ext autoreload
%autoreload 1

%aimport lib.utils

In [2]:
from lib.utils.YamlUtil import YamlUtil
import os
import sys

In [3]:
# 获取当前文件的路径
path = 'E:\\Leezed\\project\\python\\pytorch_cv_base\\test_tmp'
print(path)
cfg = YamlUtil(path, 'test_cfg.yaml')

E:\Leezed\project\python\pytorch_cv_base\test_tmp


In [4]:
cfg.debug()

{'model': {'pretrainFile1': [1, 2, 3], 'pretrainFile2': 'resnet123213'}}


In [5]:
cfg.add_root_key('model',None)

In [20]:
cfg.write_yaml()

In [9]:
cfg.add_key('model.pretrainFile1', 'resnet123213')

In [17]:
cfg.modify_key('model.pretrainFile1',[1,2,3])

In [18]:
cfg.get_value('model')

{'pretrainFile2': 'resnet123213', 'pretrainFile1': [1, 2, 3]}